In [ ]:
import os, json, random
import numpy as np
import torch

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ROOT = "."  # Colabなら基本これでOK（ノート基準）
ARTIFACT_DIR = os.path.join(ROOT, "artifacts")
IMG_DIR      = os.path.join(ROOT, "images")
LOG_DIR      = os.path.join(ROOT, "logs")
os.makedirs(ARTIFACT_DIR, exist_ok=True)
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 55.3 MB/s eta 0:00:00


In [ ]:
import joblib, numpy as np, torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
import torch.nn as nn
import torch.nn.functional as F

In [13]:
X_train = np.load(os.path.join(ARTIFACT_DIR,"X_train.npy"))
X_val   = np.load(os.path.join(ARTIFACT_DIR,"X_val.npy"))
T_train = np.load(os.path.join(ARTIFACT_DIR,"T_train.npy"))
T_val   = np.load(os.path.join(ARTIFACT_DIR,"T_val.npy"))

In [ ]:
x_train1 = torch.tensor(X_train)
t_train1 = torch.tensor(T_train)
x_val1   = torch.tensor(X_val)
t_val1   = torch.tensor(T_val)

In [ ]:
train_ds = torch.utils.data.TensorDataset(x_train1, t_train1)
val_ds   = torch.utils.data.TensorDataset(x_val1, t_val1)

In [ ]:
batch_size = 10
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader   = torch.utils.data.DataLoader(val_ds, batch_size=batch_size)

In [ ]:
class Net(pl.LightningModule):
    def __init__(self, input_dim:int, output_dim:int=3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.drop1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.drop2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(32, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        return self.fc3(x)

    def _loss_each(self, y, t):
        loss_s = F.mse_loss(y[:,0], t[:,0])
        loss_d = F.mse_loss(y[:,1], t[:,1])
        loss_p = F.mse_loss(y[:,2], t[:,2])
        return (loss_s + loss_d + loss_p)/3, loss_s, loss_d, loss_p

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss, ls, ld, lp = self._loss_each(y, t)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss, ls, ld, lp = self._loss_each(y, t)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)

input_dim = x_train1.shape[1]
net = Net(input_dim=input_dim, output_dim=t_train1.shape[1])

logger = CSVLogger(save_dir=LOG_DIR, name="exp")
trainer = pl.Trainer(max_epochs=20, accelerator="cpu", logger=logger)
trainer.fit(net, train_loader, val_loader)

torch.save(net.state_dict(), os.path.join(ARTIFACT_DIR, "model.pth"))
print("saved:", os.path.join(ARTIFACT_DIR, "model.pth"))

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


┏━━━┳━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name  ┃ Type        ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ fc1   │ Linear      │    704 │ train │     0 │
│ 1 │ bn1   │ BatchNorm1d │    128 │ train │     0 │
│ 2 │ drop1 │ Dropout     │      0 │ train │     0 │
│ 3 │ fc2   │ Linear      │  2.1 K │ train │     0 │
│ 4 │ bn2   │ BatchNorm1d │     64 │ train │     0 │
│ 5 │ drop2 │ Dropout     │      0 │ train │     0 │
│ 6 │ fc3   │ Linear      │     99 │ train │     0 │
└───┴───────┴─────────────┴────────┴───────┴───────┘

Trainable params: 3.1 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 3.1 K                                                                                                
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 7                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/_pytree.py:21: `isinstance(treespec, LeafSpec)`
is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py:317: The number of training batches 
(28) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if
you want to see logs for the training epoch.

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


saved: ./artifacts/model.pth
